# Missoula

Zoning Source: https://services.arcgis.com/HfwHS0BxZBQ1E5DY/arcgis/rest/services/Zoning_and_Land_Use_mso/FeatureServer/0/

Note, There are two versions of the RM1 district: RM1-35 and RM1-45. Whenever this ordinance refers to the 'RM1' district, the reference is referring to both the RM1-35 and RM1-45 districts.

1. Add the "OwnerParcel" layer from the parcels GDB for the city
2. Add the zoning layer

In [13]:
city = cities[index_of('Missoula')]
city

{'name': 'Missoula', 'zoning_field': 'Zoning', 'county': 'Missoula', 'residential_zones': ['R215', 'R80', 'R40', 'R20', 'RT10', 'R8', 'R5.4', 'RT5.4', 'R3', 'RT2.7', 'RM2.7', 'RM2', 'RM1.5', 'RM1', 'RM1-35', 'RM1-45', 'RM0.5', 'RMH'], 'gdb': 'cities\\Missoula.gdb', 'layer': 'cities\\Missoula.gdb\\Parcels_Missoula'}

In [14]:
codeblock = """
def eval_zoning(zone):
    if zone in ['R215', 'R80', 'R40', 'R20', 'R8', 'R5.4']:
        return 1
    elif zone in ['RT10', 'RT5.4', 'RT2.7']:
        return 2
    elif zone in ['RM2.7', 'RM2', 'RM1.5', 'RM1', 'RM1-35', 
                  'RM1-45','RM0.5', 'RMH']:
        return 5
    else:
        return 0
"""

arcpy.management.CalculateField(
    city['layer'], 
    "zoning_allows", 
    "eval_zoning(!zoning!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'Y:\\projects\\zoning_atlas\\cities\\Missoula.gdb\\Parcels_Missoula'>

In [15]:
codeblock = """
def eval_zoning(zone, lot_size):
    if zone == 'RT-10' and lot_size >= 20000:
        return 2
    elif zone == 'RT5.4' and lot_size >= 10800:
        return 2
    elif zone == 'RT2.7' and lot_size >= 5400:
        return 2
    elif zone == 'RM2.7':
        return min(max(1, math.floor(lot_size / 2700)), 5)
    elif zone == 'RM2':
        return min(max(1, math.floor(lot_size / 2000)), 5)
    elif zone == 'RM1.5':
        return min(max(1, math.floor(lot_size / 1500)), 5)
    elif zone in ['RM1', 'RM1-35', 'RM1-45', 'RHM']:
        return min(max(1, math.floor(lot_size / 1000)), 5)
    elif zone == 'RM0.5':
        return min(max(1, math.floor(lot_size / 500)), 5)
    else:
        return 1
"""

arcpy.management.CalculateField(
    city['layer'], 
    "lot_size_allows", 
    "eval_zoning(!zoning!, !lot_size!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'Y:\\projects\\zoning_atlas\\cities\\Missoula.gdb\\Parcels_Missoula'>

In [16]:
codeblock = """
def eval_zoning(zone, lot_width):
    return 5
"""

arcpy.management.CalculateField(
    city['layer'], 
    "lot_width_allows", 
    "eval_zoning(!zoning!, !lot_width!)", 
    "PYTHON3", 
    codeblock, 
    "SHORT"
)

<Result 'Y:\\projects\\zoning_atlas\\cities\\Missoula.gdb\\Parcels_Missoula'>

In [17]:
codeblock = """
def categorize(zone):
    if zone in ['RT10', 'RT5.4', 'RT2.7', 'RM2.7',
    'RM2', 'RM1.5', 'RM1', 'RM1-35', 'RM1-45', 'RM0.5', 'RMH']:
        return "Penalized Multifamily"
    elif zone in []:
        return "Multifamily"
    else:
        return "Single-Family"
"""
arcpy.management.CalculateField(
    city['layer'], 
    "category", 
    "categorize(!zoning!)",
    "PYTHON3",
    codeblock,
    "TEXT"
)

<Result 'Y:\\projects\\zoning_atlas\\cities\\Missoula.gdb\\Parcels_Missoula'>